In [3]:
from functions import *

In [4]:
# get pre-scraped text data
# evaluate information score only on relative articles

with open('rel_articles.txt', 'r') as file:
    data_code = file.read()

exec(data_code)  # Just to verify the content read


there are 12 articles in relevant_articles.


In [5]:
for article in relevant_articles:
    print(weighted_info_score(article))
# 2  2   1   1  1     
# IMO MMSI Names (people/location)

[0 0 1 1 1]
[0 0 1 1 1]
[0 0 1 1 1]
[0 0 1 1 1]
[0 0 1 1 1]
[0 0 1 1 1]
[0 0 1 1 1]
[0 0 1 1 1]
[0 0 1 1 1]
[0 0 1 1 1]
[0 0 1 1 1]
[0 0 1 1 1]


In [6]:
some_species = ['salmon', 'tuna', 'shrimp', 'crab', 'shark', 'sablefish']

fish_list = some_species
for fish in some_species:
    fish_list = list(set(fish_list).union(set(get_related_words(fish, limit = '100',topics = ['species', 'fishing']))))

#print(fish_list)

def find_words_in_list(text, word_list):
#    nlp = spacy.load('en_core_web_sm')
#    doc = nlp(text)
    fish = []
    text_words = text.split()
    found_words = []

    # Iterate over word_list and check if each word is in words_in_text
    for word in word_list:
        if word.lower() in text_words:
            found_words.append(word)

    return found_words



for article in relevant_articles:
    print("=================New Article")
    article = text_preprocess(article)
    print(find_words_in_list(article,fish_list))


fish_list = get_related_words('fish', limit = '100', topics = ['vessel', 'fishing']) - ['fish', 'fisheries','fishing']

=================New Article
['salmon', 'fish']
=================New Article
['fish', 'sablefish']
=================New Article
['fish']
=================New Article
['fish', 'mackerel']
=================New Article
['fish', 'lobster']
=================New Article
[]
=================New Article
['crab']
=================New Article
['fish']
=================New Article
['fish', 'albacore', 'tuna']
=================New Article
['yellowfin', 'fish', 'albacore', 'tuna', 'shark', 'bigeye']
=================New Article
[]
=================New Article
['fish', 'shark']


TypeError: unsupported operand type(s) for -: 'list' and 'list'

#### Thought: For a search engine, it might be a good idea to add the information score for all relevant articles from the search.

#### Challenge: It is extremely rare an article directly contains IMO number or MMSI number. We need some more types of information to check!

In [13]:
########################################################
#### find_imo: Find IMO (7-digit numbers)

def find_imo(content):    
    # regular expression for a 7-digit number (exactly 7-digit)
    imo_pattern = r'\b\d{7}\b'
    
    # find all 7-digit numbers in the content 
    imo_list = re.findall(imo_pattern, content)
    
    return imo_list

########################################################
#### find_mmsi: Find MMSI (9-digit numbers)

def find_mmsi(content):    
    # regular expression for a 9-digit number (exactly 9-digit)
    mmsi_pattern = r'\b\d{9}\b'
    
    # find all 9-digit numbers in the content 
    mmsi_list = re.findall(mmsi_pattern, content)
    
    return mmsi_list

########################################################
#### find names (company, people, etc)
# These functions require spaCy
########################################################

import spacy

########################################################
# people's names
def find_people(text):
    nlp = spacy.load('en_core_web_sm')
    doc = nlp(text)
    names = []

    for ent in doc.ents:
        if ent.label_ in ['PERSON']: # NORP is about nationality/religious/political group
            names.append(ent.text)
    return names

########################################################
# company's names
def find_company(text):
    nlp = spacy.load('en_core_web_sm')
    doc = nlp(text)
    names = []

    for ent in doc.ents:
        if ent.label_ in ['ORG']: # NORP is about nationality/religious/political group
            names.append(ent.text)
    return names


########################################################
# country's name / location
def find_location(text):
    nlp = spacy.load('en_core_web_sm')
    doc = nlp(text)
    locations = []

    for ent in doc.ents:
        if ent.label_ in ['LOC', 'GPE', 'NORP']: # location labels
            locations.append(ent.text)
    return locations


########################################################
# fishing species
def find_location(text):
    nlp = spacy.load('en_core_web_sm')
    doc = nlp(text)
    locations = []

    for ent in doc.ents:
        if ent.label_ in ['LOC', 'GPE', 'NORP']: # location labels
            locations.append(ent.text)
    return locations

########################################################
# GRT


########################################################
# Information score
def weighted_info_score(text):
    #give weigths to each information:
    # IMO: 2
    # MMSI: 2
    # Involved parties or location: 1
    # The score will be out of 10
    score_list = [find_imo, find_mmsi, find_people, find_company, find_location]
    weight_vector = np.array([2, 2, 1, 1, 1])
    score = []
    i=0
    for item in score_list:
        print(i, item(text))   
        i += 1
        if len(item(text)) > 0:
            score.append(1)
        else:
            score.append(0)
    score = np.array(score)
    weighted_score = score * weight_vector
    #np.dot(score,weight_vector)

    return weighted_score



In [14]:
import numpy as np
for article in relevant_articles:
    print(weighted_info_score(article))



0 []
1 []
2 ['Haida Gwaii', 'Miriam Maisonville', 'David Patterson', 'Adam Munro', 'Silas Levesque', 'Patterson', 'Haida Gwaii', 'Patterson', 'Levesque', 'Levesque', 'Patterson', 'Munro', 'Levesque', 'Patterson', 'Michael Andrew Bullock', 'Patterson', 'Munro', 'Patterson', 'Maisonville', 'Maisonville', 'Maisonville']
3 ['B.C.’s Supreme Court', 'ATM Fishing Ltd.', 'Munro', 'ATM', 'the Wind Walker', 'the Wind Walker', 'Munro', 'B.C. Ltd.', '605463 B.C. Ltd.', 'the Winter Wind', 'Munro', 'B.C. Ltd.', 'the Winter Wind', 'the Winter Wind', 'the Winter Wind']
4 ['Masset', 'Masset', 'B.C.', 'Masset', 'B.C.', 'Masset']
[0 0 1 1 1]
0 []
1 []
2 ['Kodiak', 'James Aaron Stevens', 'Stevens', 'Stevens', 'Stevens', 'Stevens', 'Stevens', 'Stevens', 'Kodiak', 'Stevens', 'Stevens', 'Stevens', 'Janiese Stevens', 'James', 'Stevens']
3 ['EUR', 'Office', 'the F/V Southern Seas', 'International Pacific Halibut Commission', 'IPHC', 'Alaska Department', 'KMXT', 'IFQ', 'KMXT', 'NOAA', 'Office of Law Enforcement

### Testing with NLTK
(Not really better/worse than spaCy..) Sometimes spaCy get things correctly, and sometimes not.

In [15]:
#People
import nltk
from nltk import ne_chunk, word_tokenize, pos_tag
from nltk.tree import Tree

def extract_names(text):
    names = []
    for sent in nltk.sent_tokenize(text):
        for chunk in nltk.ne_chunk(pos_tag(word_tokenize(sent))):
            if isinstance(chunk, Tree) and chunk.label() == 'PERSON':
                names.append(' '.join([token for token, pos in chunk.leaves()]))
    return names






# location
import nltk
from nltk import ne_chunk, word_tokenize, pos_tag
from nltk.tree import Tree

def extract_locations(text):
    locations = []
    for sent in nltk.sent_tokenize(text):
        for chunk in nltk.ne_chunk(pos_tag(word_tokenize(sent))):
            if isinstance(chunk, Tree) and chunk.label() == 'GPE':
                locations.append(' '.join([token for token, pos in chunk.leaves()]))
    return locations






# organization
import nltk
from nltk import ne_chunk, word_tokenize, pos_tag
from nltk.tree import Tree

def extract_organizations(text):
    organizations = []
    for sent in nltk.sent_tokenize(text):
        for chunk in nltk.ne_chunk(pos_tag(word_tokenize(sent))):
            if isinstance(chunk, Tree) and chunk.label() == 'ORGANIZATION':
                organizations.append(' '.join([token for token, pos in chunk.leaves()]))
    return organizations







def info_extraction(text):
    #give weigths to each information:
    # IMO: 2
    # MMSI: 2
    # Involved parties or location: 1
    # The score will be out of 10
    score_list = [find_imo, find_mmsi, extract_names, extract_organizations, extract_locations]
    weight_vector = np.array([2, 2, 1, 1, 1])
    score = []
    i=0
    for item in score_list:
        print(i, item(text))
        i += 1
        if len(item(text)) > 0:
            score.append(1)
        else:
            score.append(0)
    score = np.array(score)
    weighted_score = score * weight_vector
    #np.dot(score,weight_vector)

    return weighted_score

for article in relevant_articles:
    print("=================New Article")
    info_extraction(article)

=================New Article
0 []
1 []
2 ['Haida Gwaii', 'David Patterson', 'Adam Munro', 'Silas Levesque', 'Patterson', 'Haida Gwaii', 'Patterson', 'Munro', 'Masset', 'Levesque', 'Patterson', 'Munro', 'Salmon Area', 'Salmon Troll Fishery', 'Levesque', 'Patterson', 'Munro', 'Michael Andrew Bullock', 'Patterson', 'Munro', 'Munro', 'Patterson', 'Maisonville']
3 ['Supreme Court', 'Justice Miriam Maisonville', 'ATM Fishing', 'ATM', 'Wind', 'Wind', 'Wind', 'Justice Maisonville']
4 ['Masset', 'Levesque', 'Masset', 'Prince Rupert', 'Crown', 'Masset']
=================New Article
0 []
1 []
2 ['James Aaron Stevens', 'Alaska District', 'Stevens', 'Stevens', 'Alaskan Star', 'Stevens', 'Alaska Department', 'Stevens', 'Stevens', 'Stevens', 'Stevens', 'Stevens', 'Janiese Stevens', 'James', 'Law Enforcement']
3 ['EUR', 'International Pacific Halibut Commission', 'IPHC', 'USD', 'EUR', 'EUR', 'KMXT', 'IFQ', 'KMXT', 'NOAA']
4 ['Alaska', 'Kodiak', 'U.S.', 'Fish', 'Kodiak', 'Stevens', 'Stevens']
=========

#### Fish species

In [50]:
word_list = get_related_words('seafood', limit=60, topics=['fishing', 'species'])
print(word_list)

['fishing', 'fishery', 'aquaculture', 'shellfish', 'aquatic', 'seaweed', 'finfish', 'seabass', 'sealife', 'native', 'codfish', 'mariculture', 'fishmeal', 'seawater', 'fruit of the sea', 'fruits of the sea', 'seafare', 'sea vegetable', 'seafish', 'sea fish', 'seafood boil', 'shellfishery', 'food fish', 'seafoodie', 'sea stock', 'foodfish', 'chalk fish', 'sea-weed', 'green caviar', 'shellfishing', 'other fishes in the sea', 'sea fennel', 'sea onion', 'geoduck', 'saltfish', 'shorefish', 'fishfood', 'sea lettuce', 'rock samphire', 'seafolk', 'sea pumpkin', 'sea dace', 'wallfish', 'fish stock', 'edible crab', 'aquafeed', 'land shrimp', 'fishlife', 'sea snail', 'european sea bass', 'dodman', 'sea potato', 'seafooder', 'sea orb', 'bottomfish', 'tofish', 'fish food', 'seasnail', 'sea salmon', 'chicken of the sea']


In [7]:
check_word_in_list('shark', word_list)

"'shark' is not in the list."

In [15]:
get_related_words('salmon', limit=60, topics='vessel')

['redfish',
 'rockfish',
 'humpback',
 'blackfish',
 'trout',
 'sockeye',
 'salmonid',
 'smolt',
 'brown',
 'fingerling',
 'pink',
 'kipper',
 'grayling',
 'springer',
 'sparling',
 'salmon river',
 'salmonine',
 'samoun',
 'atlantic salmon',
 'salmonoid',
 'black salmon',
 'salmoniform',
 'salmon pink',
 'sea trout',
 'king-of-the-salmon',
 'silver salmon',
 'pink salmon',
 'cockfish',
 'salmon trout',
 'spring salmon',
 'rock salmon',
 'henfish',
 'salmoner',
 'coho salmon',
 'sea salmon',
 'landlocked salmon',
 'spawner',
 'wananish',
 'sockeye salmon',
 'dog salmon',
 'seatrout',
 'king salmon',
 'samlet',
 'mousefish',
 'jack salmon',
 'salmon peel',
 'land-locked salmon',
 'bull trout',
 'white salmon',
 'half-fish',
 'beaked salmon',
 'colonial salmon',
 'silver bream',
 'quisutsch',
 'foul fish',
 'salmonet',
 'mykiss',
 'humpback salmon',
 'chum salmon',
 'brown trout']

In [39]:
some_species = ['salmon', 'tuna', 'shrimp', 'crab', 'shark', 'sablefish']

In [48]:
fish_list = some_species
for fish in some_species:
    fish_list = list(set(fish_list).union(set(get_related_words(fish, limit = '100',topics = ['species', 'fishing']))))

#print(fish_list)

def find_words_in_list(text, word_list):
#    nlp = spacy.load('en_core_web_sm')
#    doc = nlp(text)
    fish = []
    text_words = text.split()
    found_words = []

    # Iterate over word_list and check if each word is in words_in_text
    for word in word_list:
        if word.lower() in text_words:
            found_words.append(word)

    return found_words



for article in relevant_articles:
    print("=================New Article")
    article = text_preprocess(article)
    print(find_words_in_list(article,fish_list))


=================New Article
['fish', 'salmon']
=================New Article
['fish', 'sablefish']
=================New Article
['fish']
=================New Article
['mackerel', 'fish']
=================New Article
['lobster', 'fish']
=================New Article
[]
=================New Article
['crab']
=================New Article
['fish']
=================New Article
['albacore', 'tuna', 'fish']
=================New Article
['albacore', 'yellowfin', 'bigeye', 'tuna', 'fish', 'shark']
=================New Article
[]
=================New Article
['fish', 'shark']


In [46]:
get_related_words('fish', limit = '100', topics = ['vessel', 'fishing'])

['fishery',
 'fisherman',
 'fishermen',
 'seafood',
 'tuna',
 'halibut',
 'lobster',
 'crab',
 'shellfish',
 'shark',
 'salmon',
 'mackerel',
 'shrimp',
 'saltwater',
 'sharks',
 'trout',
 'catfish',
 'squid',
 'swordfish',
 'freshwater',
 'lobsters',
 'gamefish',
 'dogfish',
 'bait',
 'crabs',
 'grouper',
 'crayfish',
 'meat',
 'whitebait',
 'crustaceans',
 'rockfish',
 'steelhead',
 'whitefish',
 'fillet',
 'carp',
 'sunfish',
 'flatfish',
 'seafoods',
 'finfish',
 'walleye',
 'sportfish',
 'shrimps',
 'bream',
 'goldfish',
 'salmonid',
 'sturgeon',
 'angle',
 'trouts',
 'burbot',
 'lizardfish',
 'fillets',
 'minnows',
 'tilapia',
 'mug',
 'bullheads',
 'blackfish',
 'gull',
 'sucker',
 'filets',
 'bluegill',
 'bluegills',
 'milkfish',
 'eels',
 'salmonids',
 'sticklebacks',
 'chump',
 'fool',
 'patsy',
 'soft touch',
 'steelheads',
 'brewis',
 'mark',
 'shlemiel',
 'schlemiel',
 'fall guy',
 'go fish',
 'goosefish',
 'garfish',
 'goldeye',
 'breams',
 'bloodworm',
 'tilapias',
 'nee